In [1]:
import torch
import pandas as pd
import numpy as np
import sys 
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler

sys.path.append("./")
from prediction.api import *
from prediction.data_wrapper import get_loaders

In [2]:
args.data = './data/processed/processed_cleaned_df.csv'
args.input_size = [33]
args.hidden_size = 300
args.number_of_layers = 10
args.output_size = [1]
args.test_portion = 0.2 
args.train_window = 12
args.batch_size = 64
args.learning_rate = 0.001
args.weight_decay = 0.0001
args.gpu = -1 
args.epochs = 100
args.log_freq = 30
args.debug = False
args.seed = 1
args.samples = 20
args.dropout = 0.3

In [3]:
df = pd.read_csv(args.data).iloc[:,1:]
label_index = df.columns.get_loc('0407')
print(df.head())

      GPs  month      07    0304  050103  131002     2101   010305       01  \
0  A81001      1  7271.0  4071.0  1476.0   185.0  4964.00  13032.0  69503.0   
1  A81001      2  8527.0  4071.0  1427.0   191.0  3841.00  12740.0  78753.0   
2  A81001      3  6402.0  3928.0  1278.0   198.0  4188.25  11574.0  65473.0   
3  A81001      4  7295.0  4836.0  1537.0   231.0  4535.50  13019.0  74436.0   
4  A81001      5  6671.0  5153.0  1436.0   280.0  4882.75  11824.0  65084.0   

       05  ...  030401       13           12     0106     1302       10  \
0  9767.0  ...  4310.0  59472.0   886.000000  19797.0  49930.0  15657.0   
1  9767.0  ...  4064.0  59472.0   994.666667  24304.0  49930.0  12903.0   
2  9767.0  ...  3928.0  59472.0  1103.333333  20416.0  44955.0  13236.0   
3  9767.0  ...  4836.0  58722.0  1212.000000  24713.0  45005.0  14082.0   
4  9767.0  ...  5152.0  57972.0   817.000000  20309.0  42955.0  12972.0   

    130201  050108    0603   010604  
0  41880.0   998.0  4034.0  10939.0 

In [4]:
data = df.to_numpy()
gps = data[:,0]
features = data[:,1:]

In [5]:
print(features.shape, gps.shape)

(398518, 33) (398518,)


In [6]:
total_size = len(features)
train_size = int((1-args.test_portion) * total_size)

train_data = features[:train_size]
test_data = features[train_size:]

train_gps = gps[:train_size]
test_gps = gps[train_size:]

train_unique_gps = np.unique(train_gps)
test_unique_gps = np.unique(test_gps)

scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

train_data_normalized = torch.FloatTensor(train_data_normalized)
test_data = torch.FloatTensor(test_data)

train_data_normalized, test_data

(tensor([[-1.0000, -0.9208, -0.9369,  ..., -0.9385, -0.8547, -0.9135],
         [-0.8182, -0.9071, -0.9369,  ..., -0.9032, -0.8448, -0.9123],
         [-0.6364, -0.9302, -0.9391,  ..., -0.8783, -0.9116, -0.9070],
         ...,
         [ 0.6364, -0.9437, -0.9681,  ..., -0.9603, -0.9642, -0.9361],
         [ 0.8182, -0.9443, -0.9802,  ..., -0.9591, -0.9433, -0.9594],
         [ 1.0000, -0.9438, -0.9765,  ..., -0.9736, -0.9561, -0.9516]]),
 tensor([[-1.0000, -0.9498, -0.9745,  ..., -0.9704, -0.9428, -0.9563],
         [-0.8182, -0.9569, -0.9675,  ..., -0.9558, -0.9341, -0.9126],
         [-0.6364, -0.9575, -0.9623,  ..., -0.9631, -0.9452, -0.9426],
         ...,
         [ 0.6364, -0.9620, -0.9977,  ..., -0.9920, -0.9934, -0.9938],
         [ 0.8182, -0.9668, -0.9881,  ..., -0.9858, -0.9913, -0.9929],
         [ 1.0000, -0.9567, -0.9888,  ..., -0.9916, -0.9854, -0.9877]]))

In [ ]:
def create_inout_sequences(input_data,_gps,_unique_gps,_label_index, tw):
    X = []
    Y = []
    for i in tqdm(range(len(_unique_gps))):
        for j in range(len(_gps[_gps == _unique_gps[i]]) - tw):
            seq = input_data[j:j+tw]
            label = input_data[j+tw:j+tw+1,_label_index]
            seq = np.array(seq, dtype=np.float32)
            label = np.array(label, dtype = np.float32)
            X.append(torch.from_numpy(seq).float())
            Y.append(torch.from_numpy(label).float())

    return X, Y
X_train, Y_train = create_inout_sequences(train_data, train_gps, train_unique_gps, label_index, args.train_window)
X_test, Y_test = create_inout_sequences(test_data, test_gps, test_unique_gps, label_index, args.train_window)


 58%|█████▊    | 792/1375 [00:02<00:02, 280.38it/s]

In [ ]:
X_train = torch.stack(X_train)
X_test = torch.stack(X_test)
Y_train = torch.stack(Y_train)
Y_test = torch.stack(Y_test)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

In [ ]:
def get_loaders(X, Y):
    dataset = torch.utils.data.TensorDataset(X,Y)
    return torch.utils.data.DataLoader(dataset,batch_size = args.batch_size,shuffle=True)

In [ ]:
train_loader = get_loaders(X_train, Y_train)
test_loader = get_loaders(X_test, Y_test)

In [ ]:
model = get_model(train_loader, test_loader)